In [1]:
import matplotlib.pyplot as plt
import torch
import torchvision
import data_setup
import engine
import utils
import helper_functions

from torch import nn
from torchvision import transforms
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter

device = "cuda" if torch.cuda.is_available() else 'cpu'

c:\Users\danie\GitHub\Food_Classifier\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import zipfile

from pathlib import Path

import requests

def download_data(source: str, 
                  destination: str,
                  remove_source: bool = True) -> Path:
    """Downloads a zipped dataset from source and unzips to destination.

    Args:
        source (str): A link to a zipped file containing data.
        destination (str): A target directory to unzip data to.
        remove_source (bool): Whether to remove the source after downloading and extracting.
    
    Returns:
        pathlib.Path to downloaded data.
    
    Example usage:
        download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                      destination="pizza_steak_sushi")
    """
    # Setup path to data folder
    data_path = Path("data/")
    image_path = data_path / destination

    # If the image folder doesn't exist, download it and prepare it... 
    if image_path.is_dir():
        print(f"[INFO] {image_path} directory exists, skipping download.")
    else:
        print(f"[INFO] Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)
        
        # Download pizza, steak, sushi data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"[INFO] Downloading {target_file} from {source}...")
            f.write(request.content)

        # Unzip pizza, steak, sushi data
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"[INFO] Unzipping {target_file} data...") 
            zip_ref.extractall(image_path)

        # Remove .zip file
        if remove_source:
            os.remove(data_path / target_file)
    
    return image_path

image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

[INFO] data\pizza_steak_sushi directory exists, skipping download.


WindowsPath('data/pizza_steak_sushi')

In [3]:
# Setup directories
train_dir = image_path / "train"
test_dir = image_path / "test"

# Setup ImageNet normalization levels (turns all images into similar distribution as ImageNet)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

"""
manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor,
    normalize
])
"""
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
automatic_transforms = weights.transforms()
print(f"Automatically created transforms: {automatic_transforms}")

Automatically created transforms: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


# Create data loaders

In [4]:
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transforms=automatic_transforms,
    batch_size= 32
)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x2a1a888a390>,
 ['pizza', 'steak', 'sushi'])

# Getting a pretrained model

In [5]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights= weights).to(device)

Freeze weights 

In [6]:
for param in model.features.parameters():
    param.requires_grad = False
    
helper_functions.set_seeds()

# update classifier to reflect number of class names 
model.classifier = torch.nn.Sequential(
    nn.Dropout(p = 0.2, inplace= True),
    nn.Linear(in_features= 1280, out_features= len(class_names), bias = True).to(device)
)

In [7]:
# Get a summary of the model (uncomment for full output)
"""
summary(model, 
       input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
       verbose=0,
       col_names=["input_size", "output_size", "num_params", "trainable"],
       col_width=20,
    row_settings=["var_names"]
)
"""

'\nsummary(model, \n       input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)\n       verbose=0,\n       col_names=["input_size", "output_size", "num_params", "trainable"],\n       col_width=20,\n    row_settings=["var_names"]\n)\n'

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Use SummaryWriter

In [9]:
writer = SummaryWriter()

# Edit train function in engine to use SummaryWriter

In [10]:
from typing import Dict, List
from tqdm.auto import tqdm

from engine import train_step, test_step

def train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs, device):
    # create results dictionary
    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": [],
    }
    
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device)
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device)
        
    # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )
    # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        ### Experiment Tracking ###
        # add loss results to SummaryWriter 
        writer.add_scalars(main_tag="Loss", tag_scalar_dict={"train_loss": train_loss, "test_loss": test_loss}, global_step=epoch)
        
        # add accuracy results to SummaryWriter 
        writer.add_scalars(main_tag="Accuracy", tag_scalar_dict={"train_acc": train_loss, "test_acc": test_loss}, global_step=epoch)
        
        # Track the PyTorch model architecture
        writer.add_graph(model = model, input_to_model=torch.randn(32, 3, 224, 224).to(device))
        
    #close writer 
    writer.close()
    
    return results
    

In [11]:
"""
# Train model
# Note: Not using engine.train() since the original script isn't updated to use writer
helper_functions.set_seeds()
results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=5,
                device=device)
"""

"\n# Train model\n# Note: Not using engine.train() since the original script isn't updated to use writer\nhelper_functions.set_seeds()\nresults = train(model=model,\n                train_dataloader=train_dataloader,\n                test_dataloader=test_dataloader,\n                optimizer=optimizer,\n                loss_fn=loss_fn,\n                epochs=5,\n                device=device)\n"

# Create helpuer function to build SumamryWriter()

In [12]:
def create_writer(experiment_name: str, model_name: str, extra: str = None) -> torch.utils.tensorboard.writer.SummaryWriter():
    """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir.

    log_dir is a combination of runs/timestamp/experiment_name/model_name/extra.

    Where timestamp is the current date in YYYY-MM-DD format.

    Args:
        experiment_name (str): Name of experiment.
        model_name (str): Name of model.
        extra (str, optional): Anything extra to add to the directory. Defaults to None.

    Returns:
        torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir.

    Example usage:
        # Create a writer saving to "runs/2022-06-04/data_10_percent/effnetb2/5_epochs/"
        writer = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb2",
                               extra="5_epochs")
        # The above is the same as:
        writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/effnetb2/5_epochs/")
    """
    
    from datetime import datetime
    import os
    
    # get timestamp of current date
    timestamp = datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

    if extra:
        # create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)
        
    print(f"[INFO] Created SummaryWriter and saved to: {log_dir}")
    
    return SummaryWriter(log_dir=log_dir)

In [13]:
# create an example writer

example_writer = create_writer(experiment_name="data_10_percent", model_name = "effnetb0", extra = "5 epochs")

[INFO] Created SummaryWriter and saved to: runs\2025-03-02\data_10_percent\effnetb0\5 epochs


# adjust train function to take in writer

In [14]:
from typing import Dict, List
from tqdm.auto import tqdm

from engine import train_step, test_step

def train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs, device, writer):
    # create results dictionary
    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": [],
    }
    
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device)
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device)
        
    # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )
    # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        if writer: 
        ### Experiment Tracking ###
        # add loss results to SummaryWriter 
            writer.add_scalars(main_tag="Loss", tag_scalar_dict={"train_loss": train_loss, "test_loss": test_loss}, global_step=epoch)
        
        # add accuracy results to SummaryWriter 
            writer.add_scalars(main_tag="Accuracy", tag_scalar_dict={"train_acc": train_loss, "test_acc": test_loss}, global_step=epoch)
        
        # Track the PyTorch model architecture
            writer.close()
        else:
            pass
        
    
    return results
    

In [15]:

# Download 10 percent and 20 percent training data (if necessary)
data_10_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                                     destination="pizza_steak_sushi")

data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

[INFO] data\pizza_steak_sushi directory exists, skipping download.
[INFO] data\pizza_steak_sushi_20_percent directory exists, skipping download.


In [16]:
# Setup training directory paths
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"


# Setup testing directory paths (note: use the same test dataset for both to compare the results)
test_dir = data_10_percent_path / "test"

# Check the directories
print(f"Training directory 10%: {train_dir_10_percent}")
print(f"Training directory 20%: {train_dir_20_percent}")
print(f"Testing directory: {test_dir}")

Training directory 10%: data\pizza_steak_sushi\train
Training directory 20%: data\pizza_steak_sushi_20_percent\train
Testing directory: data\pizza_steak_sushi\test


In [17]:
# create simple transforms

simple_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

In [18]:
BATCH_SIZE = 32

# Create 10% training and test DataLoaders
train_dataloader_10_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_10_percent,
    test_dir=test_dir, 
    transforms=simple_transforms,
    batch_size=BATCH_SIZE
)

# Create 20% training and test data DataLoders
train_dataloader_20_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_20_percent,
    test_dir=test_dir,
    transforms=simple_transforms,
    batch_size=BATCH_SIZE
)

In [19]:
# Create an instance of EffNetB2 with pretrained weights
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT # "DEFAULT" means best available weights
effnetb2 = torchvision.models.efficientnet_b2(weights=effnetb2_weights)

# Get the number of in_features of the EfficientNetB2 classifier layer
print(f"Number of in_features to final layer of EfficientNetB2: {len(effnetb2.classifier.state_dict()['1.weight'][0])}")

Number of in_features to final layer of EfficientNetB2: 1408


# Create Feature Extractor for pretrained models

In [20]:
def create_effnetb0():
    weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    model = torchvision.models.efficientnet_b0(weights = weights).to(device)
    
    # free base model layers
    for param in model.features.parameters():
        param.requires_grad = False
        
    model.classifier = nn.Sequential(
        nn.Dropout(p = 0.2),
        nn.Linear(in_features=1280, out_features=len(class_names)).to(device)
    )
    
    model.name = "effnetb0"
    print(f"[INFO] Created new {model.name} model")
    return model

def create_effnetb2():
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    model = torchvision.models.efficientnet_b2(weights = weights).to(device)
    
    # free base model layers
    for param in model.features.parameters():
        param.requires_grad = False
        
    model.classifier = nn.Sequential(
        nn.Dropout(p = 0.3),
        nn.Linear(in_features=1408, out_features=len(class_names)).to(device)
    )
    
    model.name = "effnetb2"
    print(f"[INFO] Created new {model.name} model")
    return model
    
    

In [21]:
effnetb0 = create_effnetb0() 
effnetb2 = create_effnetb2() 

[INFO] Created new effnetb0 model
[INFO] Created new effnetb2 model


In [22]:
num_epochs = [5, 10]
models = ["effnetb0", "effnetb2"]
train_dataloaders = {"data_10_percent" : train_dataloader_10_percent,
                     "data_20_percent": train_dataloader_20_percent}

In [23]:
helper_functions.set_seeds(seed=42)

experiment_number = 0

for dataloader_name, train_dataloader in train_dataloaders.items():
    for epochs in num_epochs:
        for model_name in models:
            experiment_number += 1
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] Model: {model_name}")
            print(f"[INFO] DataLoader: {dataloader_name}")
            print(f"[INFO] Number of epochs: {epochs}")  
            
            if model_name == "effnetb0":
                model = create_effnetb0()
            else:
                model = create_effnetb2()
                
            loss_fn = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(params=model.parameters(), lr = 0.001)
            writer = create_writer(experiment_name=dataloader_name, model_name=model_name, extra=f"{epochs}_epochs")
            
            train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs, device, writer)
            
            # save the model
            save_filepath = f"{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            utils.save_model(model = model, target_dir = "models", model_name = save_filepath)
            print("-"*50 + "\n")

[INFO] Experiment number: 1
[INFO] Model: effnetb0
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5
[INFO] Created new effnetb0 model
[INFO] Created SummaryWriter and saved to: runs\2025-03-02\data_10_percent\effnetb0\5_epochs


 20%|██        | 1/5 [00:15<01:00, 15.06s/it]

Epoch: 1 | train_loss: 0.7924 | train_acc: 0.7344 | test_loss: 0.5567 | test_acc: 0.9583


 40%|████      | 2/5 [00:30<00:45, 15.00s/it]

Epoch: 2 | train_loss: 0.4317 | train_acc: 0.9297 | test_loss: 0.3725 | test_acc: 0.9583


...

Epoch: 8 | train_loss: 0.1486 | train_acc: 0.9521 | test_loss: 0.2091 | test_acc: 0.9489


 90%|█████████ | 9/10 [02:20<00:15, 15.56s/it]

Epoch: 9 | train_loss: 0.0792 | train_acc: 0.9917 | test_loss: 0.2037 | test_acc: 0.9489


100%|██████████| 10/10 [02:35<00:00, 15.59s/it]

Epoch: 10 | train_loss: 0.0710 | train_acc: 0.9917 | test_loss: 0.2052 | test_acc: 0.9489
[INFO] Saving model to: models\effnetb2_data_20_percent_10_epochs.pth
--------------------------------------------------

